# Data Collection and Pre-Processing Lab – E-Commerce Dataset

## Step 1 – Hello, Data!

In [1]:
import pandas as pd

df_raw = pd.read_csv('data/sales_500.csv')
df_raw.head(400)

,date,customer_id,product,price,quantity,coupon_code,shipping_city
0,2024-03-13,CUST1000,Monitor,276.34,3,WELCOME15,Calgary
1,2024-03-09,CUST1001,Keyboard,662.10,1,DISCOUNT5,Toronto
2,2024-02-25,CUST1002,Headphones,609.79,4,WELCOME15,Toronto
3,2024-03-08,CUST1003,Keyboard,931.46,1,OFF20,Ottawa
4,2024-06-05,CUST1004,Keyboard,959.94,1,DISCOUNT5,Toronto
...,...,...,...,...,...,...,...
395,2024-01-07,CUST1395,Phone,1228.57,4,SAVE10,Toronto
396,2024-03-04,CUST1396,Headphones,669.92,1,SAVE10,Toronto
397,2024-03-27,CUST1397,Phone,577.88,4,NaN,Montreal
398,2024-01-18,CUST1398,Laptop,704.77,1,NaN,Ottawa


## : Merge `sales_500.csv` with Kaggle dataset

In this lab, we will also merge the original sales file (`sales_500.csv`) with a Kaggle retail dataset (`retail_sales_dataset.csv`).
We will **map Kaggle columns** to match the lab schema, then **append rows** and continue the same cleaning steps on the combined dataset.


In [2]:
# Load Kaggle dataset (make sure this file exists in the /data folder)
df_kaggle_raw = pd.read_csv('data/retail_sales_dataset.csv')

# Clean Kaggle column names
df_kaggle = df_kaggle_raw.copy()
df_kaggle.columns = [c.strip().lower().replace(' ', '_') for c in df_kaggle.columns]

# Map Kaggle columns -> Lab columns
# Kaggle columns: date, customer_id, product_category, quantity, price_per_unit, total_amount
df_kaggle_mapped = pd.DataFrame({
    'date': df_kaggle.get('date'),
    'customer_id': df_kaggle.get('customer_id'),
    'product': df_kaggle.get('product_category'),
    'price': df_kaggle.get('price_per_unit'),
    'quantity': df_kaggle.get('quantity'),
    'coupon_code': pd.NA,           # not available in Kaggle dataset
    'shipping_city': pd.NA          # not available in Kaggle dataset
})

# Align columns (union) and append rows
all_cols = sorted(set(df_raw.columns).union(set(df_kaggle_mapped.columns)))
df_raw_aligned = df_raw.reindex(columns=all_cols)
df_kaggle_aligned = df_kaggle_mapped.reindex(columns=all_cols)

df_raw = pd.concat([df_raw_aligned, df_kaggle_aligned], ignore_index=True)

print('✅ Merge complete!')
print('Combined dataset shape:', df_raw.shape)
df_raw.head()


✅ Merge complete!
Combined dataset shape: (1500, 7)


,coupon_code,customer_id,date,price,product,quantity,shipping_city
0,WELCOME15,CUST1000,2024-03-13,276.34,Monitor,3,Calgary
1,DISCOUNT5,CUST1001,2024-03-09,662.10,Keyboard,1,Toronto
2,WELCOME15,CUST1002,2024-02-25,609.79,Headphones,4,Toronto
3,OFF20,CUST1003,2024-03-08,931.46,Keyboard,1,Ottawa
4,DISCOUNT5,CUST1004,2024-06-05,959.94,Keyboard,1,Toronto


## Step 2 – Pick the Right Container
A dictionary is appropriate because it allows fast key-based lookups 
(e.g., customer_id → transactions) and flexible mutation during cleaning.
Namedtuples are immutable and better suited for fixed schemas, while sets
are useful only for uniqueness checks, not structured records.

## Step 3 – Implement Functions and Data Structure

In [3]:
class TransactionCleaner:
    def __init__(self, df):
        self.df = df.copy()

    def clean(self):
        self.df = self.df.dropna(subset=['price', 'quantity'])
        self.df = self.df[self.df['quantity'] > 0]
        return self.df

    def total(self):
        return (self.df['price'] * self.df['quantity']).sum()

cleaner = TransactionCleaner(df_raw)
df_step3 = cleaner.clean()
cleaner.total()

np.float64(1396632.29)

## Step 4 – Bulk Loaded

In [4]:
transactions_dict = df_step3.to_dict(orient='records')
transactions_dict[:499]

[{'coupon_code': 'WELCOME15',
  'customer_id': 'CUST1000',
  'date': '2024-03-13',
  'price': 276.34,
  'product': 'Monitor',
  'quantity': 3,
  'shipping_city': 'Calgary'},
 {'coupon_code': 'DISCOUNT5',
  'customer_id': 'CUST1001',
  'date': '2024-03-09',
  'price': 662.1,
  'product': 'Keyboard',
  'quantity': 1,
  'shipping_city': 'Toronto'},
 {'coupon_code': 'WELCOME15',
  'customer_id': 'CUST1002',
  'date': '2024-02-25',
  'price': 609.79,
  'product': 'Headphones',
  'quantity': 4,
  'shipping_city': 'Toronto'},
 {'coupon_code': 'OFF20',
  'customer_id': 'CUST1003',
  'date': '2024-03-08',
  'price': 931.46,
  'product': 'Keyboard',
  'quantity': 1,
  'shipping_city': 'Ottawa'},
 {'coupon_code': 'DISCOUNT5',
  'customer_id': 'CUST1004',
  'date': '2024-06-05',
  'price': 959.94,
  'product': 'Keyboard',
  'quantity': 1,
  'shipping_city': 'Toronto'},
 {'coupon_code': 'OFF20',
  'customer_id': 'CUST1005',
  'date': '2024-06-14',
  'price': 87.05,
  'product': 'Phone',
  'quantity

## Step 5 – Quick Profiling

In [5]:
min_price = df_step3['price'].min()
mean_price = df_step3['price'].mean()
max_price = df_step3['price'].max()

unique_cities = set(df_step3['shipping_city'])
min_price, mean_price, max_price, len(unique_cities)

(np.float64(25.0), np.float64(373.92601333333334), np.float64(1499.58), 6)

## Step 6 – Spot the Grime
Missing values, zero/negative quantities, and inconsistent coupon codes.

## Step 7 – Cleaning Rules

In [6]:
before_rows = len(df_raw)
df_clean = df_raw.dropna(subset=['price', 'quantity'])
df_clean = df_clean[df_clean['quantity'] > 0]
after_rows = len(df_clean)
before_rows, after_rows

(1500, 1500)

## Step 8 – Transformations

In [7]:
def parse_discount(code):
    if pd.isna(code):
        return 0
    digits = ''.join(filter(str.isdigit, str(code)))
    return int(digits) if digits else 0

df_clean['discount_percent'] = df_clean['coupon_code'].apply(parse_discount)
df_clean[['coupon_code', 'discount_percent']].head()

,coupon_code,discount_percent
0,WELCOME15,15
1,DISCOUNT5,5
2,WELCOME15,15
3,OFF20,20
4,DISCOUNT5,5


## Step 9 – Feature Engineering

In [8]:
df_clean['order_date'] = pd.to_datetime(df_clean['date'])
latest_date = df_clean['order_date'].max()
df_clean['days_since_purchase'] = (latest_date - df_clean['order_date']).dt.days

## Step 10 – Mini-Aggregation

In [9]:
df_clean['revenue'] = df_clean['price'] * df_clean['quantity']
revenue_by_city = df_clean.groupby('shipping_city')['revenue'].sum().to_dict()
list(revenue_by_city.items())[:5]

[('Calgary', 161850.34),
 ('Montreal', 184951.42),
 ('Ottawa', 217806.6),
 ('Toronto', 198684.23),
 ('Vancouver', 177339.7)]

## Step 11 – Serialization Checkpoint

In [10]:
df_clean.to_csv('data/cleaned_sales.csv', index=False)
df_clean.to_json('data/cleaned_sales.json', orient='records', indent=2)

C:\Users\kevin\AppData\Local\Temp\ipykernel_2300\3923981207.py:2: Pandas4Warning: The default 'epoch' date format is deprecated and will be removed in a future version, please use 'iso' date format instead.
  df_clean.to_json('data/cleaned_sales.json', orient='records', indent=2)


## Step 12 – Soft Interview Reflection
Functions and classes helped modularize the data-cleaning logic, making the workflow reusable, readable, and easier to maintain.

## Data Dictionary
| Field | Type | Description | Source |
|------|------|------------|--------|
| date | Date | Order date | Primary CSV |
| customer_id | String | Customer identifier | Primary CSV |
| product | String | Product name | Primary CSV |
| price | Float | Unit price | Primary CSV |
| quantity | Integer | Quantity purchased | Primary CSV |
| coupon_code | String | Promo code | Primary CSV |
| discount_percent | Integer | Parsed discount | Derived |
| shipping_city | String | Delivery city | Primary CSV |
| days_since_purchase | Integer | Days since order | Synthetic |